In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

from datasets import Dataset, DatasetDict, concatenate_datasets, load_dataset

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [ ]:
# 1. Model ve Tokenizer'ı Yükle
model_name = "SmolLM2-360M-Instruct-v1-model"  # Mistral 7B modeli
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [ ]:
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})


In [ ]:
# 2. Tapaco Veri Setini Yükleyin
dataset = load_dataset("turkish-nlp-suite/InstrucTurca", split="train", cache_dir="/media/hosman/Yedek/Datasets/")  # Tapaco veri setini yükle
dataset2 = load_dataset("Metin/WikiRAG-TR", split="train", cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"question": "Input", "answer": "Output"})
dataset3 = load_dataset("kayrab/patient-doctor-qa-tr-167732",  cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"question_content": "Input", "question_answer": "Output"})

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

from datasets import Dataset, DatasetDict, concatenate_datasets, load_dataset

In [ ]:
dataset,dataset2,dataset3 

In [ ]:
dataset3 = concatenate_datasets([dataset3["train"], dataset3["test"]])

In [ ]:
dataset = concatenate_datasets([dataset, dataset2, dataset3])

In [ ]:
dataset

In [ ]:
from datasets import Dataset
import numpy as np

# None içeren satırları temizleyen fonksiyon
def remove_none_rows(example):
    return example["Input"] is not None and example["Output"] is not None

# None değerleri içeren satırları filtrele
dataset = dataset.filter(remove_none_rows)

In [ ]:
from datasets import Dataset
import numpy as np

# Token uzunluklarını hesaplayan fonksiyon
def get_token_lengths(example):
    input_length = len(example["Input"])
    output_length = len(example["Output"])
    return {"input_length": input_length, "output_length": output_length}

# Tüm veri seti için hesaplama
token_lengths = dataset.map(get_token_lengths, batched=False)

# Maksimum ve ortalama token sayıları
max_input_length = max(token_lengths["input_length"])
max_output_length = max(token_lengths["output_length"])

avg_input_length = np.mean(token_lengths["input_length"])
avg_output_length = np.mean(token_lengths["output_length"])

# Sonuçları yazdır
print(f"📌 Maksimum Input Token Sayısı: {max_input_length}")
print(f"📌 Ortalama Input Token Sayısı: {avg_input_length:.2f}")
print(f"📌 Maksimum Output Token Sayısı: {max_output_length}")
print(f"📌 Ortalama Output Token Sayısı: {avg_output_length:.2f}")


In [ ]:
# Filtreleme fonksiyonu
def filter_long_samples(example):
    input_length = len(example["Input"])
    output_length = len(example["Output"])
    
    # Eğer input ve output ortalamadan büyükse filtrele (False döndür)
    return not (input_length > 128 and output_length > 128)

# Yeni filtrelenmiş dataset
dataset = dataset.filter(filter_long_samples)

# Filtrelenmiş veri kümesi hakkında bilgi
print(f"✅ Filtrelenmiş veri seti satır sayısı: {len(dataset)}")


In [ ]:
from datasets import DatasetDict

# 1. Veriyi train ve test olarak ayırma
# Örneğin, dataset zaten tek bir büyük veri seti (örneğin "data") içeriyor
# Bunu %80 train ve %20 test olarak bölelim
train_dataset, temp_dataset = dataset.train_test_split(test_size=0.2, seed=42).values()

# 2. Test setini de %50 validation ve %50 test olarak bölelim
val_dataset, test_dataset = temp_dataset.train_test_split(test_size=0.2, seed=42).values()

# 3. Veriyi tokenizasyon işlemi için tokenize edelim
def tokenize_function(example):
    # Input ve Output'u tokenize et
    input_tokens = tokenizer(example["Input"], padding="max_length", truncation=True, max_length=128)
    output_tokens = tokenizer(example["Output"], padding="max_length", truncation=True, max_length=128)

    # Tokenized Input ve Output'u döndür
    return {
        "input_ids": input_tokens["input_ids"],
        "attention_mask": input_tokens["attention_mask"],
        "labels": output_tokens["input_ids"]
    }

# 4. Tokenize işlemini her bir split için uygulayalım
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True).remove_columns(["Input","Output"])
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True).remove_columns(["Input","Output"])
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True).remove_columns(["Input","Output"])

# Tokenize edilmiş veri setlerini birleştirebilirsiniz (opsiyonel)
final_dataset = DatasetDict({
    'train': tokenized_train_dataset,
    'validation': tokenized_val_dataset,
    'test': tokenized_test_dataset
})



In [ ]:
tokenized_test_dataset[0]["input_ids"], tokenized_test_dataset[0]["attention_mask"]

In [ ]:
tokenized_test_dataset

In [ ]:
#model.gradient_checkpointing_enable()
#model.use_cache = False


In [ ]:
import math
import numpy as np
import wandb
import evaluate
from sentence_transformers import SentenceTransformer, util
from collections import Counter

# Metrikleri yükle
bleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
chrf = evaluate.load("chrf")
#bert_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Distinct-N hesaplayan fonksiyon
def compute_distinct_n(preds, n=2):
    all_ngrams = [tuple(preds[i:i+n]) for i in range(len(preds)-n+1)]
    return len(set(all_ngrams)) / max(1, len(all_ngrams))

# Metrikleri hesaplayan ana fonksiyon
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    results = {}

    try:
        loss = np.mean(preds)
        results["perplexity"] = math.exp(loss)
    except Exception as e:
        results["perplexity"] = f"error: {str(e)}"

    try:
        results["bleu"] = bleu.compute(predictions=preds, references=labels)["score"]
    except Exception as e:
        results["bleu"] = f"error: {str(e)}"

    try:
        rouge_scores = rouge.compute(predictions=preds, references=labels)
        results.update({
            "rouge-1": rouge_scores["rouge1"].mid.fmeasure,
            "rouge-2": rouge_scores["rouge2"].mid.fmeasure,
            "rouge-L": rouge_scores["rougeL"].mid.fmeasure,
        })
    except Exception as e:
        results["rouge-1"] = results["rouge-2"] = results["rouge-L"] = f"error: {str(e)}"

    try:
        results["chrf"] = chrf.compute(predictions=preds, references=labels)["score"]
    except Exception as e:
        results["chrf"] = f"error: {str(e)}"

    try:
        results["distinct-1"] = compute_distinct_n(preds, n=1)
        results["distinct-2"] = compute_distinct_n(preds, n=2)
    except Exception as e:
        results["distinct-1"] = results["distinct-2"] = f"error: {str(e)}"

    # try:
    #     similarities = [util.pytorch_cos_sim(bert_model.encode(p), bert_model.encode(l)).item() for p, l in zip(preds, labels)]
    #     results["semantic_similarity"] = sum(similarities) / len(similarities)
    # except Exception as e:
    #     results["semantic_similarity"] = f"error: {str(e)}"

    # WandB loglama
    wandb.log(results)
    return results


In [ ]:
import wandb
from transformers import TrainerCallback
import torch

class WandBQuestionCallback(TrainerCallback):
    def __init__(self, tokenizer, model, questions, device="cuda", log_interval=1000):
        self.tokenizer = tokenizer
        self.model = model.to(device)
        self.device = device
        self.questions = questions  # List of question strings
        self.log_interval = log_interval
        
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % self.log_interval == 0:
            wandb.log({"step": state.global_step})
            self.log_model_responses()

    def log_model_responses(self):
        responses = {}
        for question in self.questions:
            messages = [
                {"role": "system", "content": "Sen yardımsever bir asistansın"},
                {"role": "user", "content": question}
            ]
            input_text = self.tokenizer.apply_chat_template(messages, tokenize=False)
            inputs = self.tokenizer.encode(input_text, return_tensors="pt").to(self.device)
            outputs = self.model.generate(inputs, max_new_tokens=128, temperature=0.2, top_p=0.9, do_sample=True)
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            responses[question] = response
        
        wandb.log({"model_responses": responses})

In [ ]:
# Örnek sorular
questions = [
    "433 * b - 7420490 = -7413995 denklemini çöz.",
    "Türkiye'nin başkenti neresidir?",
    "E=mc^2 denkleminin fiziksel anlamı nedir?",
    "Merhaba.Nasılsın?",
    "Merhaba, dün diş çekimi yapıldıktan sonra bu sabah aşırı kanama ile hekime başvurdum. Pihtinin oluştuğunu, ancak kanamanın durmadığı gerekçesiyle dikiş işlemi uyguladı. Bugün herhangi bir kanama veya ağrı yok, yalnız dikiş bölgesinde mukusa benzer bir doku oluştu. Tekrar gitmem gerekir mi?",
    "Merhaba, ben 18 yaşındayım, geçen yıl elimin üst kısmı yanmıştı, şimdi iyileşti ancak elimin üstünde yanık izi kaldı. Bu iz için herhangi bir ilaç veya farklı tedavi yöntemi var mıdır?"
    "Mulan filminin hikayesi hangi kaynaktan esinlenmiştir?",
    "Kartografya günümüzde nasıl teknolojilerden faydalanıyor?"

]

# Callback'i oluştur
wandb_callback = WandBQuestionCallback(tokenizer, model, questions)

In [ ]:
# 4. Eğitim Ayarlarını Tanımlayın
training_args = TrainingArguments(
    output_dir="./SmolLM2-360M-Instruct-v1",  # Çıktı dizini
    evaluation_strategy="epoch",         # Değerlendirme adımları
    save_strategy="steps",               # Kaydetme adımları
    save_steps=300,                      # Her 500 adımda modeli kaydet
    logging_dir="./logs",                # Log dosyaları dizini
    logging_steps=500,                   # Her 100 adımda log yazdır
    learning_rate=2e-5,                  # Öğrenme oranı
    num_train_epochs=6,                  # Epoch sayısı
    per_device_train_batch_size=32,       # GPU başına batch boyutu
    per_device_eval_batch_size=32,       # GPU başına batch boyutu
    gradient_accumulation_steps=4,       # Gradient birikimi için adım sayısı
    bf16=True,                           # 16-bit floating-point
    fp16_opt_level="O2",                # Optimizasyon düzeyi (O1, O2, O3)
    dataloader_num_workers=4,
    #evaluation_strategy="no",           # Sadece eğitim (değerlendirme yapılmıyor)
    report_to="wandb",                    # WandB veya diğer araçlara raporlama yok
    save_total_limit=2,                  # Sadece son iki checkpoint'i sakla
    lr_scheduler_type="cosine",  # Cosine learning rate decay
    warmup_steps=1000,           # İlk 1000 adımda LR'yi yavaş yavaş artır
    weight_decay=0.01,           # AdamW kullanırken weight decay ekle
    optim="adamw_torch",         # Daha hızlı AdamW optimizasyonu
    
    
)

# 5. Trainer Nesnesi ile Eğitimi Başlatın
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_dataset["train"],
    eval_dataset=final_dataset["validation"],
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics,
    callbacks=[wandb_callback]
)

In [ ]:
trainer.train(resume_from_checkpoint="SmolLM2-360M-Instruct-v1/checkpoint-6300")

 70%|███████   | 123382/175782 [14:06:39<5:47:32,  2.51it/s]

In [ ]:
# 6. Eğitilmiş Modeli Kaydedin
model.save_pretrained("./SmolLM2-360M-Instruct-v1")
tokenizer.save_pretrained("./SmolLM2-360M-Instruct-v1")

print("Eğitim tamamlandı ve model kaydedildi.")